<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_projects/blob/main/predict_judge/Predict_Jugment_automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 0: loss, 1:win

In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 25.6 MB/s eta 0:00:00


In [2]:
pip install --upgrade wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=ec239d59df16d3d0e94f936c5cfa1d379699c67c46dbf881597d4fe1867a034e
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [3]:
import wandb

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd "/content/drive/MyDrive/Data/judge"

/content/drive/MyDrive/Data/judge


In [7]:
import warnings
warnings.filterwarnings('ignore')

import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    AdamW,
    get_linear_schedule_with_warmup,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from datasets import load_metric

In [8]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
print('--train--')
print(train_df.shape)
display(train_df.head())
print('--test--')
print(test_df.shape)
display(test_df.head())

--train--
(2478, 5)


,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


--test--
(1240, 4)


,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."


In [10]:
train_df["contents"] = train_df["first_party"] + "," + train_df["second_party"] + "," + train_df["facts"]
test_df["contents"] = test_df["first_party"] + "," + test_df["second_party"] + "," + test_df["facts"]

In [12]:
from sklearn.model_selection import train_test_split

X = train_df['contents'].values.tolist()
y = train_df['first_party_winner'].values.tolist()

train_texts, val_texts, train_labels, val_labels  = train_test_split(X, y, test_size=.2, stratify=y)

In [13]:
from tokenizers import Tokenizer

In [14]:
# MODEL_NAME =  'distilbert-base-uncased'
# MODEL_NAME = 'roberta-base'
# MODEL_NAME =  'google/electra-base-discriminator'
MODEL_NAME = 'nlpaueb/legal-bert-base-uncased'

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [16]:
# tokenizer.tokenize(train_texts[0])

In [17]:
# test_texts = test_df['facts'].values.tolist()
test_texts = test_df['contents'].values.tolist()

In [18]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [19]:
class JudgeDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels=None):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    if self.labels:
      item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings["input_ids"])

In [20]:
train_dataset = JudgeDataset(train_encodings, train_labels)
val_dataset = JudgeDataset(val_encodings, val_labels)
test_dataset = JudgeDataset(test_encodings)

In [21]:
random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.cuda.empty_cache()  # Clear the GPU cache
torch.cuda.set_per_process_memory_fraction(0.8)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,  num_labels=2)
model.to(device)

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [22]:
def compute_metrics_fn(eval_preds):
  metrics = dict()

  accuracy_metric = load_metric('accuracy')

  logits = eval_preds.predictions
  labels = eval_preds.label_ids
  preds = np.argmax(logits, axis=-1)

  metrics.update(accuracy_metric.compute(predictions=preds, references=labels))

  return metrics

In [23]:
wandb.init()
wandb.finish()
# Learning_rate = 0.000192447022729965
# Weight_decay = 0.1
# Lr_scheduler = 'cosine'
BATCH_SIZE=8
num_epochs =10



training_args = TrainingArguments(
    output_dir ='./results',
    # learning_rate= Learning_rate,
    num_train_epochs=num_epochs,  # number of training epochs
    per_device_train_batch_size=BATCH_SIZE, # batch size of training
    per_device_eval_batch_size=BATCH_SIZE, # batch size of evaluation
    gradient_accumulation_steps = 4,
    # weight_decay=Weight_decay,
    fp16=True,
    # lr_scheduler_type = Lr_scheduler,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    load_best_model_at_end=True,


)

# # Define the ModelCheckpoint callback
# checkpoint_callback = ModelCheckpoint(
#     dirpath='./models',                # Directory where the best model will be saved
#     filename='bert_best_model',              # Name of the checkpoint files
#     monitor='accuracy',                # Metric to monitor
#     mode='max',                         # The metric should be minimized (e.g., 'min' for loss, 'max' for accuracy)
#     save_top_k=1,                       # Save the best model
#     save_weights_only=True,             # Save only the model weights, not the entire model
# )
trainer = Trainer(
    model=model,                        # model
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training data
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics_fn,
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: solowosophia13. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.655600,0.647453,0.665323
2,0.651900,0.652986,0.665323
3,0.643700,0.643957,0.665323
4,0.645500,0.638251,0.665323
5,0.639000,0.640627,0.665323
6,0.641300,0.637410,0.665323
7,0.640700,0.638628,0.665323
8,0.638600,0.637410,0.665323
9,0.643000,0.637534,0.665323
10,0.638700,0.637468,0.665323


TrainOutput(global_step=620, training_loss=0.6437973822316816, metrics={'train_runtime': 559.1606, 'train_samples_per_second': 35.446, 'train_steps_per_second': 1.109, 'total_flos': 5214861117235200.0, 'train_loss': 0.6437973822316816, 'epoch': 10.0})

In [24]:
# Make prediction
raw_pred, _, _ = trainer.predict(test_dataset)
# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

In [25]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [26]:
submit = pd.read_csv('./sample_submission.csv')

In [27]:
submit['first_party_winner'] = y_pred
submit.to_csv('./legalparty_submit.csv', index=False)
print('Done')

Done
